In [1]:
import plotly.express as px
import pandas as pd
basepath = 'your_path_here'

In [2]:
dfw_prop_df = pd.read_pickle(basepath + "Data/OnTime/PropagateSummary.pkl")
dfw_air_coord = pd.read_csv(basepath + "Data/OnTime/dfw_airport_coordinates.csv")
dfw_prop_df['Year'] = dfw_prop_df['FlightDate'].apply(lambda x : int(x[0:4])).copy()
dfw_prop_df['Month'] = dfw_prop_df['FlightDate'].apply(lambda x : int(x[5:7])).copy()
dfw_prop_df['Day'] = dfw_prop_df['FlightDate'].apply(lambda x : int(x[8:])).copy()
dfw_prop_df["Orig"] = dfw_prop_df['OrigList'].apply(lambda x : x[0])
dfw_prop_df["Dest"] = dfw_prop_df['DestList'].apply(lambda x : x[0])

In [3]:
dfw_gt_1 = dfw_prop_df[dfw_prop_df['Flights'] > 1 ]
dfw_eq_1 = dfw_gt_1[dfw_gt_1['Flights'] == 1 ]
dfw_coord = dfw_air_coord[['Dest', 'lat_prp', 'lon_prp']]

Create vis with chlorpleth based on counts of DEST where weather delay out of DFW

In [4]:
# Original Weather Delay
# Group by year counting flights for each Dest
dfw_orig = dfw_prop_df[dfw_prop_df['Orig'] == 'DFW']
dfw_Dest_cnts = dfw_orig.groupby(['Dest']).size().to_frame('count')

dfw_dest_coord = dfw_Dest_cnts.merge(dfw_coord, how = 'inner', on='Dest', sort=True)
# Destinations Where DFW encounters Weather Delays
fig = px.scatter_geo(dfw_dest_coord, lat='lat_prp', lon='lon_prp', color='count', scope='usa', range_color= (1, 250), hover_data={'lon_prp': False, 'lat_prp': False},
                     color_continuous_scale='reds', hover_name='Dest', height = 600, width = 600)
fig.write_html(basepath + "Code/VisualWork/DFW_DestDelaysMap.html")
fig.write_image(basepath + "Code/VisualWork/DFW_DestDelaysMap.png")
fig#.show()

In [7]:

dfw_orig = dfw_prop_df[dfw_prop_df['Orig'] == 'DFW']

prop_cnts_by_dest = pd.DataFrame()
for prop in range(0, dfw_orig['PropagateCount'].max() + 1,1):
    dfw_le_prop_cnt = dfw_orig[dfw_orig['PropagateCount'] >= prop]
    dfw_le_prop_cnt["Dest"] = dfw_le_prop_cnt['DestList'].apply(lambda x : x[prop])
    dfw_Dest_cnts = dfw_le_prop_cnt.groupby(['Dest']).size().to_frame('count')
    dfw_dest_coord = dfw_Dest_cnts.merge(dfw_coord, how = 'inner', on='Dest', sort=True)
    dfw_dest_coord.insert(0, 'Propagation Number', prop)
    prop_cnts_by_dest = pd.concat([prop_cnts_by_dest,dfw_dest_coord], axis= 0)

prop_to = prop_cnts_by_dest[(prop_cnts_by_dest['Propagation Number'] > 0) & (prop_cnts_by_dest['Propagation Number'] < 5)]
# Using non-DFW airports as the max counts or DFW causes the color in the chlorpleth
#    to be exaggerated. DFW counts are over 3000. The remaining are 103 or less.
prop_not_dfw  = prop_to[prop_to['Dest'] != 'DFW']
max_cnt = prop_not_dfw['count'].max()
# Airports Delays Propagated to
fig = px.scatter_geo(prop_to, lat='lat_prp', lon='lon_prp', color='count', scope='usa', range_color= (1, max_cnt), hover_data={'lon_prp': False, 'lat_prp': False},
                     color_continuous_scale='reds', hover_name='Dest', facet_col = 'Propagation Number', facet_col_wrap=2, facet_row_spacing= 0.04, facet_col_spacing= 0.04,
                     height = 800, width = 800)
fig.write_html(basepath + "Code/VisualWork/DFW_PropDelaysMap.html")
fig.write_image(basepath + "Code/VisualWork/DFW_PropDelaysMap.png")
fig

/var/folders/nk/k2s_bx5j2kz5nkdhcg5y4llr0000gn/T/ipykernel_48376/2101126628.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/nk/k2s_bx5j2kz5nkdhcg5y4llr0000gn/T/ipykernel_48376/2101126628.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/nk/k2s_bx5j2kz5nkdhcg5y4llr0000gn/T/ipykernel_48376/2101126628.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d